In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [7]:
torch.version.cuda


In [8]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# print(device)


In [9]:
print(device)


cpu


In [10]:
# ufc_df = pd.read_csv("ufc_combined_0905.csv", index_col=0)
# ufc_df = pd.read_csv("ufc_combined_0911.csv", index_col=0)
ufc_df = pd.read_csv("fight_stats_diff_920.csv", index_col=0)
# ufc_df.drop('Time', axis=1, inplace=True)
ufc_df.head()


,Winner,Ht Diff,Reach Diff,Career W Diff,Career L Diff,W Diff,L Diff,Num Fights Diff,W Perc Diff,Sig Strikes Avg Diff,...,KO Opp Avg Diff,SUB Opp Avg Diff,CTRL Avg Diff,CTRL Opp Avg Diff,Time Avg Diff,Streak Diff,Age Diff,Career Fights Diff,Career W Perc Diff,Stance Diff
0,1,-2,0,-4,-2,1,0,1,100,2.000000,...,0.000000,0.000000,11.000000,0.000000,58.000000,1,-9,-6.0,15.384615,-3
1,0,-5,-4,3,4,1,3,4,20,28.600000,...,0.200000,0.000000,53.000000,-648.400000,-287.200000,-1,3,7.0,-11.904762,0
2,1,2,2,-4,-2,0,1,1,-50,57.500000,...,0.000000,0.000000,42.000000,302.500000,702.000000,0,-1,-6.0,3.809524,3
3,0,2,-3,2,-2,-1,-2,-3,16,29.500000,...,-0.333333,-0.166667,32.166667,239.333333,421.166667,3,1,0.0,12.500000,1
4,1,-2,-3,1,0,1,1,2,0,-31.583333,...,0.000000,0.333333,191.583333,-209.500000,-239.250000,3,0,1.0,1.307190,-1


In [11]:
# split the dataset
X, y = ufc_df.drop("Winner", axis=1), ufc_df["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [12]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(6110, 36) (1528, 36) (6110,) (1528,)


In [13]:
print(y_train.mean(), y_test.mean())


0.5042553191489362 0.49476439790575916


In [14]:
ufc_df.head()

,Winner,Ht Diff,Reach Diff,Career W Diff,Career L Diff,W Diff,L Diff,Num Fights Diff,W Perc Diff,Sig Strikes Avg Diff,...,KO Opp Avg Diff,SUB Opp Avg Diff,CTRL Avg Diff,CTRL Opp Avg Diff,Time Avg Diff,Streak Diff,Age Diff,Career Fights Diff,Career W Perc Diff,Stance Diff
0,1,-2,0,-4,-2,1,0,1,100,2.000000,...,0.000000,0.000000,11.000000,0.000000,58.000000,1,-9,-6.0,15.384615,-3
1,0,-5,-4,3,4,1,3,4,20,28.600000,...,0.200000,0.000000,53.000000,-648.400000,-287.200000,-1,3,7.0,-11.904762,0
2,1,2,2,-4,-2,0,1,1,-50,57.500000,...,0.000000,0.000000,42.000000,302.500000,702.000000,0,-1,-6.0,3.809524,3
3,0,2,-3,2,-2,-1,-2,-3,16,29.500000,...,-0.333333,-0.166667,32.166667,239.333333,421.166667,3,1,0.0,12.500000,1
4,1,-2,-3,1,0,1,1,2,0,-31.583333,...,0.000000,0.333333,191.583333,-209.500000,-239.250000,3,0,1.0,1.307190,-1


In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Copying the data
# X_train_scaled = X_train.copy()
# X_test_scaled = X_test.copy()

# # List of continuous columns
# conts = X.drop(cats, axis=1).columns.values.tolist()

# # Convert the continuous columns to float
# X_train_scaled[conts] = X_train_scaled[conts].astype(float)
# X_test_scaled[conts] = X_test_scaled[conts].astype(float)

# # Scaling the continuous columns
# scaler = StandardScaler()
# X_train_scaled.loc[:, conts] = scaler.fit_transform(X_train[conts])
# X_test_scaled.loc[:, conts] = scaler.transform(X_test[conts])



In [16]:
class UfcDataset(Dataset):
    # def __init__(self, df, cats, conts, targets):
    def __init__(self, df, cats, targets):
        self.X_cats = df[cats].astype(np.int64).values
        # self.X_conts = df[conts].astype(np.float32).values
        self.y = targets.astype(np.float32).values.reshape(-1, 1)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, i):
        # return [self.X_cats[i], self.X_conts[i], self.y[i]]
        return [self.X_cats[i], self.y[i]]


In [19]:
# train_dataset = UfcDataset(X_train_scaled, cats, conts, y_train)
# test_dataset = UfcDataset(X_test_scaled, cats, conts, y_test)

cats = X.columns.values.tolist()
print(cats)

train_dataset = UfcDataset(X_train_scaled, cats, y_train)
test_dataset = UfcDataset(X_test_scaled, cats, y_test)


['Ht Diff', 'Reach Diff', 'Career W Diff', 'Career L Diff', 'W Diff', 'L Diff', 'Num Fights Diff', 'W Perc Diff', 'Sig Strikes Avg Diff', 'Sig Str % Diff', 'Sig Strikes Opp Avg Diff', 'Sig Str % Opp Diff', 'Strikes Avg Diff', 'Str % Diff', 'Strikes Opp Avg Diff', 'Str % Opp Diff', 'TD Avg Diff', 'TD % Diff', 'TD Opp Avg Diff', 'TD % Opp Diff', 'KD Avg Diff', 'KD Opp Avg Diff', 'DEC Avg Diff', 'KO Avg Diff', 'SUB Avg Diff', 'DEC Opp Avg Diff', 'KO Opp Avg Diff', 'SUB Opp Avg Diff', 'CTRL Avg Diff', 'CTRL Opp Avg Diff', 'Time Avg Diff', 'Streak Diff', 'Age Diff', 'Career Fights Diff', 'Career W Perc Diff', 'Stance Diff']


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [33]:
# Data loaders
bz=32
# bz = 64
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=bz, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=bz, shuffle=False)


In [34]:
# Get the number of categorical and continuous features from the dataset
num_cats = train_dataset.X_cats.shape[1]  # Number of categorical columns
num_conts = train_dataset.X_conts.shape[1]  # Number of continuous columns

# Calculate the total number of features
num_features = num_cats + num_conts

print(f"Number of categorical features: {num_cats}")
print(f"Number of continuous features: {num_conts}")
print(f"Total number of features: {num_features}")


Number of categorical features: 7
Number of continuous features: 71
Total number of features: 78


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network class
class BinaryClassificationNet(nn.Module):
    def __init__(self, emb_dims, num_conts, fc_layer_sizes, emb_drop, ps):
        super(BinaryClassificationNet, self).__init__()

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in emb_dims])
        self.emb_drop = nn.Dropout(emb_drop)

        # Continuous feature layers
        self.bn_cont = nn.BatchNorm1d(num_conts)

        # Fully connected layers
        all_layer_sizes = [sum([nf for ni, nf in emb_dims]) + num_conts] + fc_layer_sizes
        self.layers = nn.ModuleList()
        for i in range(len(all_layer_sizes)-1):
            self.layers.append(nn.Linear(all_layer_sizes[i], all_layer_sizes[i+1]))
            self.layers.append(nn.ReLU(inplace=True))
            self.layers.append(nn.Dropout(ps[i]))

        # Output layer
        self.output = nn.Linear(all_layer_sizes[-1], 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_cats, x_conts):
        # Assuming x_cats is your categorical input tensor
        # print([x_cat.max().item() for x_cat in x_cats.T])

        # Embedding part
        x_cat_emb = [emb_layer(x_cat) for emb_layer, x_cat in zip(self.embeddings, x_cats.T)]
        x_cat_emb = torch.cat(x_cat_emb, 1)
        x_cat_emb = self.emb_drop(x_cat_emb)

        # Continuous features
        x_conts = self.bn_cont(x_conts)

        # Concatenate embeddings and continuous features
        x = torch.cat([x_cat_emb, x_conts], 1)

        # Fully connected layers
        for layer in self.layers:
            x = layer(x)

        # Output layer
        x = self.output(x)
        return self.sigmoid(x)

# Example usage
input_size = num_features  # For example, if you have 10 features
# model = BinaryClassificationNet(input_size)

# Adjust embedding dimensions to reflect the actual number of categories
# emb_dims = [(1950, 10),  # For the first feature, with max value 1949
#             (1907, 10),  # For the second feature, with max value 1906
#             (15, 6),     # For the third feature, with max value 14
#             (4, 3),      # For the fourth feature, with max value 3
#             (5, 3),      # For the fifth feature, with max value 4
#             (156, 8),    # For the sixth feature, with max value 155
#             (680, 10)]   # For the seventh feature, with max value 679

# num_conts = 71  # Number of continuous features
# fc_layer_sizes = [100, 50]  # Example fully connected layer sizes
# emb_drop = 0.1  # Example dropout rate for embeddings
# ps = [0.5, 0.5]  # Dropout rates for fully connected layers
print(len(cats))

# emb_dims = [(len(ufc_df[cat].unique()), min(50, len(ufc_df[cat].unique())//2)) for cat in cats]
# feature_importance = [1.0]*(len(cats)-2)  + [0.1] + [0.1]# Placeholder for feature importance
# emb_dims = [(len(ufc_df[cat].unique()), min(100, max(5, int(np.log2(len(ufc_df[cat].unique())) * imp))))
#             for cat, imp in zip(cats, feature_importance)]
emb_dims = [(len(ufc_df[cat].unique()), min(50, max(1, int(np.log2(len(ufc_df[cat].unique())))))) for cat in cats]

num_conts = len(conts)  # 71 in your case
fc_layer_sizes = [256, 64, 16]
# fc_layer_sizes = [512, 256, 64] #, 16]  # Adding more capacity and an additional layer

emb_drop = 0.5
ps = [0.5] * 3

# emb_drop = 0.3  # Slightly lower dropout for embedding layers
# ps = [0.6, 0.6, 0.6]  # Progressive dropout rates for fully connected layers

# Initialize the model
model = BinaryClassificationNet(emb_dims, num_conts, fc_layer_sizes, emb_drop, ps)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Example training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in train_dl:  # Assuming train_dl is your DataLoader
        x_cats, x_conts, labels = batch  # Unpack the three values from the batch
        optimizer.zero_grad()  # Zero out gradients

        # Forward pass (make sure your model accepts both categorical and continuous inputs)
        outputs = model(x_cats, x_conts)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing/Evaluating the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation during evaluation
    correct = 0
    total = 0
    for batch in test_dl:  # Assuming test_dl is your DataLoader
        x_cats, x_conts, labels = batch  # Unpack categorical, continuous features, and labels
        x_cats, x_conts, labels = x_cats.to(device), x_conts.to(device), labels.to(device)

        # Forward pass
        outputs = model(x_cats, x_conts)

        # Convert outputs to binary predictions (for binary classification)
        preds = (outputs > 0.5).float()  # Threshold at 0.5 for binary classification

        total += labels.size(0)
        correct += (preds == labels).sum().item()  # Count correct predictions

    # Calculate and print accuracy
    print(f"Accuracy: {100 * correct / total:.2f}%")


7
Epoch [1/5], Loss: 0.7313
Epoch [2/5], Loss: 0.7262
Epoch [3/5], Loss: 0.6437
Epoch [4/5], Loss: 0.7529
Epoch [5/5], Loss: 0.6422
Accuracy: 59.75%


In [36]:
# Save the model parameters after testing
model_save_path = 'ufc_model_6014.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Model saved to ufc_model_6014.pth
